In [241]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier


from sklearn.ensemble import VotingClassifier


import joblib

In [242]:
df=pd.read_csv("C:\\Users\\admin\\Desktop\\python\\heart.csv")

In [243]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Age                           303 non-null    int64  
 1   Sex                           303 non-null    int64  
 2   Chest_Pain                    303 non-null    int64  
 3   Resting_Blood_Pressur         303 non-null    int64  
 4   Cholesterol                   303 non-null    int64  
 5   Fasting_Blood_Sugar           303 non-null    int64  
 6   Electrocardiographic_Results  303 non-null    int64  
 7   Heart_Rate                    303 non-null    int64  
 8   Exercise_Induced_Angina       303 non-null    int64  
 9   ST_Depression                 303 non-null    float64
 10  Slope                         303 non-null    int64  
 11  Ca                            303 non-null    int64  
 12  Thal                          303 non-null    int64  
 13  Targe

In [244]:
df.isnull().sum()

Age                             0
Sex                             0
Chest_Pain                      0
Resting_Blood_Pressur           0
Cholesterol                     0
Fasting_Blood_Sugar             0
Electrocardiographic_Results    0
Heart_Rate                      0
Exercise_Induced_Angina         0
ST_Depression                   0
Slope                           0
Ca                              0
Thal                            0
Target                          0
dtype: int64

In [245]:
df.duplicated().sum()

np.int64(1)

In [246]:
df.sample(4)

,Age,Sex,Chest_Pain,Resting_Blood_Pressur,Cholesterol,Fasting_Blood_Sugar,Electrocardiographic_Results,Heart_Rate,Exercise_Induced_Angina,ST_Depression,Slope,Ca,Thal,Target
39,65,0,2,160,360,0,0,151,0,0.8,2,0,2,1
125,34,0,1,118,210,0,1,192,0,0.7,2,0,2,1
174,60,1,0,130,206,0,0,132,1,2.4,1,2,3,0
144,76,0,2,140,197,0,2,116,0,1.1,1,0,2,1


In [247]:
df["Sex"]=df["Sex"].map({0:"Male",1:"Female"})

In [248]:
df["Sex"].sample(3)

142      Male
281    Female
291    Female
Name: Sex, dtype: object

In [249]:
x=df.iloc[:,0:13]

In [250]:
x.sample()

,Age,Sex,Chest_Pain,Resting_Blood_Pressur,Cholesterol,Fasting_Blood_Sugar,Electrocardiographic_Results,Heart_Rate,Exercise_Induced_Angina,ST_Depression,Slope,Ca,Thal
110,64,Male,0,180,325,0,1,154,1,0.0,2,0,2


In [251]:
x.shape

(303, 13)

In [252]:
y=df.iloc[:,-1]

In [253]:
y.shape

(303,)

In [254]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)

In [255]:
tlf1=ColumnTransformer(transformers=[


    ("onehotencoding",OneHotEncoder(drop="first",sparse_output=False,dtype="int"),[1]),
    

],remainder="passthrough")

In [256]:
x_train_one=tlf1.fit_transform(x_train)
x_test_one=tlf1.transform(x_test)

In [257]:
ss=StandardScaler()
x_train_ss=ss.fit_transform(x_train_one)
x_test_ss=ss.fit_transform(x_test_one)

In [258]:
clf1=LogisticRegression()

clf3=RandomForestClassifier(n_estimators=300,
    max_depth=15,
    min_samples_split=4,
    min_samples_leaf=2,
    max_features='sqrt',
    bootstrap=True,
    class_weight='balanced',
    random_state=42
)
clf4=SVC(probability=True)
clf5=XGBClassifier(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    
    eval_metric='logloss'
)
clf2=GradientBoostingClassifier()



In [259]:
estemators=[("lr",clf1),("gd",clf2),("rf",clf3),("svc",clf4),("xgboost",clf5)]


In [260]:
for i in estemators:
    x=cross_val_score(i[1],x_train_ss,y_train,cv=5,scoring="accuracy")
    print(i[0],np.mean(x))

lr 0.7970048309178744
gd 0.7663768115942029
rf 0.8102415458937198
svc 0.797487922705314
xgboost 0.8191304347826087


In [261]:
vo=VotingClassifier(estimators=estemators,voting="soft")

In [262]:
x=np.mean(cross_val_score(vo,x_train_ss,y_train,cv=5,scoring="accuracy"))

In [263]:
vo.fit(x_train_ss,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('gd', GradientBoostingClassifier()),
                             ('rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_depth=15,
                                                     min_samples_leaf=2,
                                                     min_samples_split=4,
                                                     n_estimators=300,
                                                     random_state=42)),
                             ('svc', SVC(probability=True)),
                             ('xgboost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=...
                                            feature_weights=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.05, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=4,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None, ...))],
                 voting='soft')

In [264]:
svc = SVC(probability=True)

In [265]:
x_pred=vo.predict(x_test_ss)

In [266]:
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix

In [267]:
accuracy_score(x_pred,y_test)

0.868421052631579

In [268]:
precision_score(x_pred,y_test)

0.926829268292683

In [269]:
confusion_matrix(x_pred,y_test)

array([[28,  3],
       [ 7, 38]])

In [270]:
joblib.dump(tlf1, "transformer.joblib")

['transformer.joblib']

In [271]:
joblib.dump(ss, "standardscaler.joblib")

['standardscaler.joblib']

In [272]:
joblib.dump(vo,"model.joblib")

['model.joblib']

In [273]:
save_model=joblib.load("model.joblib")

In [274]:
save_model

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('gd', GradientBoostingClassifier()),
                             ('rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_depth=15,
                                                     min_samples_leaf=2,
                                                     min_samples_split=4,
                                                     n_estimators=300,
                                                     random_state=42)),
                             ('svc', SVC(probability=True)),
                             ('xgboost',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=...
                                            feature_weights=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.05, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=4,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=200, n_jobs=None,
                                            num_parallel_tree=None, ...))],
                 voting='soft')